In [5]:
!pip install pyspark findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, collect_list, concat_ws,col, floor, when, concat, array, expr, regexp_replace
from functools import reduce

spark = SparkSession.builder.appName("mj").getOrCreate()

In [10]:
## reading csv
albums=spark.read.format("csv").option("header",True).option("inferschema",True).load('/content/spotify-albums_data_2023.csv')
albums.show()
track=spark.read.csv('/content/spotify_tracks_data_2023.csv',header=True,inferSchema=True)
track.show()

+--------------------+--------------------+------------+-----------+----------+-------+------------+--------------------+--------------------+-------------------+----------------+--------------------+--------------------+--------------+-------------------+-------------+----------------+--------+--------+--------+--------+--------+--------+---------+---------+------------+
|          track_name|            track_id|track_number|duration_ms|album_type|artists|total_tracks|          album_name|        release_date|              label|album_popularity|            album_id|           artist_id|      artist_0|           artist_1|     artist_2|        artist_3|artist_4|artist_5|artist_6|artist_7|artist_8|artist_9|artist_10|artist_11|duration_sec|
+--------------------+--------------------+------------+-----------+----------+-------+------------+--------------------+--------------------+-------------------+----------------+--------------------+--------------------+--------------+--------------

In [12]:
## droping dulicates
dedup_albums = albums.dropDuplicates()
dropped_null = dedup_albums.dropna(subset=['track_name','track_id','artist_id'])

dedup_tracks = track.dropDuplicates()
drop_null = dedup_tracks.dropna(subset=['id'])

dropped_null.show()
drop_null.show()

+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+-----------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+------------+
|          track_name|            track_id|track_number|duration_ms| album_type|artists|total_tracks|          album_name|        release_date|               label|album_popularity|            album_id|           artist_id|          artist_0|   artist_1|artist_2|artist_3|artist_4|artist_5|artist_6|artist_7|artist_8|artist_9|artist_10|artist_11|duration_sec|
+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+-----------+--------+--------+--------

In [13]:
## merging all the featured artists columns in featured_artists column

featured_artist = dropped_null.withColumn("featured_artists_array", array([f"artist_{i}" for i in range(1, 12)])) \
       .withColumn("featured_artists", expr("concat_ws(', ', filter(featured_artists_array, x -> x is not null))")) \
       .drop(*[f"artist_{i}" for i in range(1, 12)], "featured_artists_array")\

#featured_artist.orderBy(col("duration_ms").desc()).show()
featured_artist.show()


+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+----------------+
|          track_name|            track_id|track_number|duration_ms| album_type|artists|total_tracks|          album_name|        release_date|               label|album_popularity|            album_id|           artist_id|          artist_0|duration_sec|featured_artists|
+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+----------------+
|Jack Is Dead - Or...|4ZmOXj5x6ER0DSQyD...|           1|     408712|     single|   NULL|           3|Deep House Sessio...|2012-03-15 00:00:...|Purple Gorilla Re...|               0|

In [17]:
## renaming artist_0 column
featured_artist = featured_artist.withColumnRenamed("artist_0","main_artist")
featured_artist.show()

+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+----------------+
|          track_name|            track_id|track_number|duration_ms| album_type|artists|total_tracks|          album_name|        release_date|               label|album_popularity|            album_id|           artist_id|       main_artist|duration_sec|featured_artists|
+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+----------------+
|Jack Is Dead - Or...|4ZmOXj5x6ER0DSQyD...|           1|     408712|     single|   NULL|           3|Deep House Sessio...|2012-03-15 00:00:...|Purple Gorilla Re...|               0|

In [ ]:
''' Convert ms to total seconds
total_seconds = floor((col("duration_ms").cast("integer") / 1000).cast("integer"))

# Extract minutes and seconds
minutes = floor(total_seconds / 60).cast("integer")
seconds = floor(total_seconds % 60).cast("integer")

# Format seconds with leading zero
formatted_seconds = when(seconds < 10, lit("0") + seconds.cast("string")) \
                    .otherwise(seconds.cast("string"))
#formatted_seconds = floor(formatted_seconds)

# Combine into MM:SS format
featured_artist = featured_artist.withColumn(
    "duration",
    concat(minutes.cast("string"), lit(":"), formatted_seconds.cast("string"))
).withColumn("formatted_seconds", seconds)
featured_artist.show()
'''

+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+----------------+--------+-------------+-----------------+
|          track_name|            track_id|track_number|duration_ms| album_type|artists|total_tracks|          album_name|        release_date|               label|album_popularity|            album_id|           artist_id|       main_artist|duration_sec|featured_artists|duration|seconds_debug|formatted_seconds|
+--------------------+--------------------+------------+-----------+-----------+-------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+----------------+--------+-------------+-----------------+
|Jack Is Dead - Or...|4ZmOXj5x6ER0DSQyD...|           1|  

In [23]:
## droping extra cols
clean_album=featured_artist.drop('artists','duration_sec')
clean_album.show()

+--------------------+--------------------+------------+-----------+-----------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+----------------+
|          track_name|            track_id|track_number|duration_ms| album_type|total_tracks|          album_name|        release_date|               label|album_popularity|            album_id|           artist_id|       main_artist|featured_artists|
+--------------------+--------------------+------------+-----------+-----------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+----------------+
|Jack Is Dead - Or...|4ZmOXj5x6ER0DSQyD...|           1|     408712|     single|           3|Deep House Sessio...|2012-03-15 00:00:...|Purple Gorilla Re...|               0|0pJsDp2HZWcjscjqj...|04rOJJh4mLlRvlO4W...|    Purple Gorilla|          

In [21]:
# cleaning tracks as required
clean_track=track.filter((col("track_popularity") > 50) & (col("explicit")=='false'))
clean_track.show()

+--------------------+----------------+--------+
|                  id|track_popularity|explicit|
+--------------------+----------------+--------+
|3vR4dOzdMZMeDSGRX...|              51|   false|
|0yebM7lJyoX5doGOC...|              51|   false|
|5kMvu3XFFvHsb7Fle...|              52|   false|
|4CHIJuqR1jLfzA72T...|              52|   false|
|5tLDX5Z50VZAvga56...|              52|   false|
|4EVwzagXAg5p4zYm4...|              52|   false|
|5GJ5uZJiDboTiTPpN...|              52|   false|
|3PHCRVk2PdPjhv2EU...|              52|   false|
|2XoSLsS75B0BF6CCY...|              52|   false|
|24pIscrDqVkkAp9KH...|              52|   false|
|5Ny8dTK7mUgpL04RV...|              52|   false|
|5WDpxxOIYliyYQ4HS...|              53|   false|
|0boRS2EhpwbEvN6TW...|              53|   false|
|4kxDa9niTJIyh89af...|              53|   false|
|2iQzGiBMP2V6F3his...|              53|   false|
|1bVLy04fBPi2JnRpX...|              53|   false|
|0vKGvQdkbesI7RGKX...|              54|   false|
|6O6ZLs5VuJ6wOSiIB..

In [29]:
# adding radio_mix column and sorting popularity high to low
radio_mix=clean_album.join(clean_track,clean_album.track_id==clean_track.id,"inner").withColumn("radio_mix", lit("true"))\
.orderBy(col("track_popularity").desc())
radio_mix.show()

+--------------------+--------------------+------------+-----------+-----------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+-----------------+--------------------+--------------------+----------------+--------+---------+
|          track_name|            track_id|track_number|duration_ms| album_type|total_tracks|          album_name|        release_date|               label|album_popularity|            album_id|           artist_id|      main_artist|    featured_artists|                  id|track_popularity|explicit|radio_mix|
+--------------------+--------------------+------------+-----------+-----------+------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+-----------------+--------------------+--------------------+----------------+--------+---------+
|All I Want For Ch...|0bYg9bo50gSsH3LtX...|           2|     241

In [ ]:
# changing column positions, selecting columns
selected_cols = radio_mix.select("track_name","track_id","track_number","album_name",
                                 "album_id","album_type","duration_ms","total_tracks","release_date","label",
                                 "track_popularity","album_popularity","artist_id","main_artist","featured_artists",
                                 "explicit","radio_mix")

In [31]:
selected_cols.show()

+--------------------+--------------------+------------+--------------------+--------------------+-----------+-----------+------------+--------------------+--------------------+----------------+----------------+--------------------+-----------------+--------------------+--------+---------+
|          track_name|            track_id|track_number|          album_name|            album_id| album_type|duration_ms|total_tracks|        release_date|               label|track_popularity|album_popularity|           artist_id|      main_artist|    featured_artists|explicit|radio_mix|
+--------------------+--------------------+------------+--------------------+--------------------+-----------+-----------+------------+--------------------+--------------------+----------------+----------------+--------------------+-----------------+--------------------+--------+---------+
|All I Want For Ch...|0bYg9bo50gSsH3LtX...|           2|     Merry Christmas|61ulfFSmmxMhc2wCd...|      album|     241106|     

In [ ]:
# writing to bigquery
selected_cols.write.format("bigquery") \
    .option("table", "project:dataset.table") \
    .option("temporaryGcsBucket", "temp-bucket-name") \
    .mode("overwrite") \
    .save()